# (Optional) Link G-drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import libraries

In [ ]:
!pip install hyperspy[all]
!git clone --branch final https://github.com/poyentung/unmix.git
from IPython.display import clear_output 
clear_output()


import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'unmix'))

from utils import normalisation as norm 
from utils import visualisation as visual
from utils.load import SEMDataset
from src.utils import same_seeds
from src.dim_reduction import Experiment
from models.autoencoder import AutoEncoder
from src.segmentation import PixelSegmenter
from py_widgets import widgets

%load_ext autoreload
%autoreload 2

# Load files

## Test file

In [ ]:
# locate the file path
file_path = 'unmix/tutorial/test.bcf'

# Load .bcf file
sem = SEMDataset(file_path)

# Set the elemental peaks in the dataset
sem.set_feature_list(['Al_Ka', 'C_Ka', 'Ca_Ka', 'Fe_Ka', 'K_Ka', 'O_Ka', 'Si_Ka', 'Ti_Ka', 'Zn_La'])

Set feature_list to ['Al_Ka', 'C_Ka', 'Ca_Ka', 'Fe_Ka', 'K_Ka', 'O_Ka', 'Si_Ka', 'Ti_Ka', 'Zn_La']


## Upload file

In [ ]:
from google.colab import files
uploaded = files.upload()

file_path=list(uploaded.keys())[0]
sem = SEMDataset(file_path)

# Dataset preprocessing

## View the dataset


In [ ]:
widgets.view_bcf_dataset(sem)

## Process the dataset

In [ ]:
# Rebin both edx and bse dataset
sem.rebin_signal(size=(2,2))

# normalisation to make the spectrum of each pixel summing to 1.
sem.peak_intensity_normalisation()

# Remove the first peak until the energy of 0.1 keV
sem.remove_fist_peak(end=0.1) 

# Denoise the X-ray profile using PCA.
sem.peak_denoising_PCA(n_components_to_reconstruct=10, plot_results=False)

In [ ]:
# View the dataset (bse, edx etc.) at one time.
widgets.view_bcf_dataset(sem)

## Normalisation

In [ ]:
# Normalise the dataset using these three methods.
sem.normalisation([norm.neighbour_averaging, 
                   norm.zscore, 
                   norm.softmax])

In [ ]:
widgets.view_pixel_distributions(sem, 
                                norm_list=[norm.neighbour_averaging,
                                           norm.zscore,
                                           norm.softmax], 
                                peak='Fe_Ka', 
                                cmap='inferno')

## Assign RGB to elemental peaks

In [ ]:
widgets.view_rgb(sem)

In [ ]:
print('After normalisation:')
widgets.view_intensity_maps(edx=sem.normalised_elemental_data, element_list=sem.feature_list)